In [43]:
# imports
import pandas as pd
import numpy as np
from keras import backend as K
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer

In [117]:
# retrieve data
test = pd.read_csv('/kaggle/input/equipfails/equip_failures_test_set.csv', na_values='na')
train = pd.read_csv('/kaggle/input/equipfails/equip_failures_training_set.csv', na_values='na')

# handle nan in test
test = test.interpolate(axis=1, limit=10, limit_direction='both').fillna(0)
train = train.interpolate(axis=1, limit=10, limit_direction='both').fillna(0); # ';' to suppress output

In [118]:
train.columns

Index(['id', 'target', 'sensor1_measure', 'sensor2_measure', 'sensor3_measure',
       'sensor4_measure', 'sensor5_measure', 'sensor6_measure',
       'sensor7_histogram_bin0', 'sensor7_histogram_bin1',
       ...
       'sensor105_histogram_bin2', 'sensor105_histogram_bin3',
       'sensor105_histogram_bin4', 'sensor105_histogram_bin5',
       'sensor105_histogram_bin6', 'sensor105_histogram_bin7',
       'sensor105_histogram_bin8', 'sensor105_histogram_bin9',
       'sensor106_measure', 'sensor107_measure'],
      dtype='object', length=172)

In [119]:
train.describe()

,id,target,sensor1_measure,sensor2_measure,sensor3_measure,sensor4_measure,sensor5_measure,sensor6_measure,sensor7_histogram_bin0,sensor7_histogram_bin1,...,sensor105_histogram_bin2,sensor105_histogram_bin3,sensor105_histogram_bin4,sensor105_histogram_bin5,sensor105_histogram_bin6,sensor105_histogram_bin7,sensor105_histogram_bin8,sensor105_histogram_bin9,sensor106_measure,sensor107_measure
count,60000.000000,60000.000000,6.000000e+04,6.000000e+04,6.000000e+04,6.000000e+04,6.000000e+04,6.000000e+04,6.000000e+04,6.000000e+04,...,6.000000e+04,6.000000e+04,6.000000e+04,6.000000e+04,6.000000e+04,6.000000e+04,6.000000e+04,6.000000e+04,6.000000e+04,6.000000e+04
mean,30000.500000,0.016667,5.933650e+04,1.218892e+08,3.362355e+08,6.080781e+06,4.570921e+03,2.891619e+03,1.987528e+03,3.473278e+03,...,4.489584e+05,2.161615e+05,4.470912e+05,3.948278e+05,3.335662e+05,3.455772e+05,1.393022e+05,9.364293e+03,8.582815e+02,8.454228e+02
std,17320.652413,0.128020,1.454301e+05,3.390767e+08,7.767583e+08,8.665659e+07,7.385434e+04,8.349924e+04,1.021688e+05,1.269912e+05,...,1.359823e+06,8.355753e+05,1.283248e+06,1.203716e+06,1.123527e+06,1.739849e+06,4.825416e+05,1.028744e+05,2.701536e+04,2.682815e+04
min,1.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,15000.750000,0.000000,8.340000e+02,1.200000e+01,1.800000e+01,1.200000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.710000e+03,1.050000e+03,2.341500e+03,3.001955e+03,4.200000e+02,9.400000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,30000.500000,0.000000,3.077600e+04,1.562600e+04,1.800000e+02,8.600000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,2.288310e+05,1.093380e+05,2.160700e+05,1.846520e+05,8.610700e+04,3.893200e+04,3.354000e+03,0.000000e+00,0.000000e+00,0.000000e+00
75%,45000.250000,0.000000,4.866800e+04,3.088725e+04,1.454000e+03,4.280000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,4.359295e+05,2.170020e+05,4.637035e+05,4.007340e+05,2.729930e+05,1.667065e+05,1.373765e+05,1.966000e+03,0.000000e+00,0.000000e+00
max,60000.000000,1.000000,2.746564e+06,1.065657e+09,2.130707e+09,8.584298e+09,1.329241e+07,1.635745e+07,1.942250e+07,2.248755e+07,...,1.340369e+08,1.172823e+08,1.005277e+08,8.377309e+07,6.701847e+07,1.195801e+08,3.350924e+07,1.675462e+07,2.708070e+06,2.708070e+06


In [120]:
train.head()

,id,target,sensor1_measure,sensor2_measure,sensor3_measure,sensor4_measure,sensor5_measure,sensor6_measure,sensor7_histogram_bin0,sensor7_histogram_bin1,...,sensor105_histogram_bin2,sensor105_histogram_bin3,sensor105_histogram_bin4,sensor105_histogram_bin5,sensor105_histogram_bin6,sensor105_histogram_bin7,sensor105_histogram_bin8,sensor105_histogram_bin9,sensor106_measure,sensor107_measure
0,1.0,0.0,76698.0,1.065392e+09,2.130706e+09,280.0,0.0,0.0,0.0,0.0,...,1240520.0,493384.0,721044.0,469792.0,339156.0,157956.0,73224.0,0.0,0.0,0.0
1,2.0,0.0,33058.0,1.652900e+04,0.000000e+00,0.0,0.0,0.0,0.0,0.0,...,421400.0,178064.0,293306.0,245416.0,133654.0,81140.0,97576.0,1500.0,0.0,0.0
2,3.0,0.0,41040.0,2.063400e+04,2.280000e+02,100.0,0.0,0.0,0.0,0.0,...,277378.0,159812.0,423992.0,409564.0,320746.0,158022.0,95128.0,514.0,0.0,0.0
3,4.0,0.0,12.0,0.000000e+00,7.000000e+01,66.0,0.0,10.0,0.0,0.0,...,240.0,46.0,58.0,44.0,10.0,0.0,0.0,0.0,4.0,32.0
4,5.0,0.0,60874.0,3.112100e+04,1.368000e+03,458.0,0.0,0.0,0.0,0.0,...,622012.0,229790.0,405298.0,347188.0,286954.0,311560.0,433954.0,1218.0,0.0,0.0


In [121]:
# trim cols to measures and target
cols = [col for col in cols if 'measure' in col]
train = train[cols + ['target']]
test = test[cols]

In [122]:
# split into input (X) and output (Y) variables
X = train.drop(columns='target').values
Y = train['target'].values
X_test = test.values

In [123]:
# standardize the input feature
sc = StandardScaler()
scaler = sc.fit(X)
X = scaler.transform(X)
X_test = scaler.transform(X_test)

In [124]:
# oversampling the failures to overcome class imbalance
sm = SMOTE(random_state=42)
X, Y = sm.fit_resample(X, Y)
X_test

array([[ 0.05192574,  2.78257194,  2.31022414, ..., -0.13992824,
        -0.0317704 , -0.03151279],
       [ 0.21856393, -0.35927528, -0.43281516, ..., -0.03888876,
        -0.0317704 , -0.03151279],
       [ 1.09735796, -0.35893879, -0.43268592, ...,  0.10446626,
        -0.0317704 , -0.03151279],
       ...,
       [-0.0583825 , -0.35940057, -0.43287269, ..., -0.13979572,
        -0.0317704 , -0.03151279],
       [-0.40768047,  2.78247337,  2.31022413, ..., -0.14015676,
        -0.0317704 , -0.03151279],
       [ 0.00456237,  2.78256178,  2.31022413, ..., -0.13982928,
        -0.0317704 , -0.03151279]])

In [125]:
# def metrics
def recall_metric(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_metric(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_metric(y_true, y_pred):
    precision = precision_metric(y_true, y_pred)
    recall = recall_metric(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [126]:
classifier = Sequential()

classifier.add(Dense(50, activation='relu', kernel_initializer='random_normal', input_dim=100)) # Input Layer
classifier.add(Dense(40, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dense(30, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dense(20, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dense(10, activation='relu', kernel_initializer='random_normal'))
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal')) # Output Layer

classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', f1_metric]);

In [127]:
#Fitting the data to the training dataset
# using mini-batch of 50
# 15 epochs to prevent over-fitting on small dataset
classifier.fit(X,Y, batch_size=50, epochs=10)

Epoch 1/10
118000/118000 [==============================] - 5s 44us/step - loss: 0.1590 - accuracy: 0.9586 - f1_metric: 0.9584
Epoch 2/10
118000/118000 [==============================] - 5s 42us/step - loss: 0.0929 - accuracy: 0.9732 - f1_metric: 0.9728
Epoch 3/10
118000/118000 [==============================] - 5s 41us/step - loss: 0.0725 - accuracy: 0.9786 - f1_metric: 0.9784
Epoch 4/10
118000/118000 [==============================] - 5s 41us/step - loss: 0.0589 - accuracy: 0.9821 - f1_metric: 0.9819
Epoch 5/10
118000/118000 [==============================] - 5s 41us/step - loss: 0.0495 - accuracy: 0.9847 - f1_metric: 0.9846
Epoch 6/10
118000/118000 [==============================] - 5s 40us/step - loss: 0.0433 - accuracy: 0.9867 - f1_metric: 0.9865
Epoch 7/10
118000/118000 [==============================] - 5s 40us/step - loss: 0.0389 - accuracy: 0.9878 - f1_metric: 0.9876
Epoch 8/10
118000/118000 [==============================] - 5s 41us/step - loss: 0.0354 - accuracy: 0.9892 - f1

In [128]:
# predict on test data
y_pred = classifier.predict(X_test)
y_pred

array([[0.0000000e+00],
       [6.8545341e-06],
       [9.1327429e-03],
       ...,
       [0.0000000e+00],
       [3.1002462e-03],
       [2.4063766e-08]], dtype=float32)

In [129]:
# format dataframe to fit submission specification and write csv
submit = pd.DataFrame(y_pred)
submit[0] = submit[0].astype(int)
submit.index += 1
submit.rename(columns={0: 'target'});

,target
1,0
2,0
3,0
4,0
5,0
...,...
15997,0
15998,0
15999,0
16000,0


In [130]:
submit.shape

(16001, 1)

In [131]:
submit.to_csv('submission.csv', index_label='id', header=['target'])